In [34]:
import tensorflow
import time
from scipy.io import wavfile
from IPython.display import Audio
from IPython.core.display import display
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

#!pip install librosa
import librosa
import librosa.display
#from folder_locations import  data_folder, bird_train, bird_test, human_folder, human_clips_folder, warblr_folder

import os
#import plotly.express as px
#import sweetviz as sv
from sklearn.utils import shuffle
import pickle

random_number = 93

In [35]:
#locations
#data_folder = "C:/Users/bobtu/Desktop/University/Thesis/Ellie/Data/"
ssd = os.getcwd()[:2]+"//"
warblr_folder = ssd+"warblr/" #+ data-folder
wav_folder = warblr_folder +"wav/"
human_folder = ssd+"cv-corpus-7.0-2021-07-21/en/"
hclip_folder = human_folder + "clips/"

In [36]:
list_of_wav_names = os.listdir(wav_folder) # a list of every filename in the folder (not the data, just the filename)

list_of_mp3_names = os.listdir(hclip_folder)

In [ ]:
#list_of_mp3_names

In [ ]:
#df_yesbirdy = pd.read_csv('D:/Code/yesbirdy.csv')

In [ ]:
#ssd[:2]

In [37]:
#read the csv file of warblr
df_birdy = pd.read_csv(warblr_folder+'warblrb10k_public_metadata.csv')

In [38]:
#when a bird is heard = 1
df_yesbirdy = df_birdy[df_birdy['hasbird'] == 1]
#when not = 0
df_nobirdy = df_birdy[df_birdy['hasbird'] == 0]

In [39]:
#reset index and drop old index
df_yesbirdy = df_yesbirdy.reset_index(drop=True)

In [40]:
#get sample rate per sound sample
sr = librosa.get_samplerate(warblr_folder + 'wav/000e088a-69f7-4d7a-ba7b.wav')

#create function that retrieves data and sample rate from file
def add_bird_data(row):
    
    bird, _ = librosa.load(warblr_folder + 'wav/' + row + ".wav", sr)
    
    return bird

#create function that retrieves data and sample rate from file
def create_bird_dict(df):
    new_dict = {}
    for i in range(len(df)):
        new_dict[df['itemid'][i]] = add_bird_data(df['itemid'][i])
    
    return new_dict

In [ ]:
sr

In [41]:
#applying the function add_bird_data for every row
birdy_dict = create_bird_dict(df_yesbirdy)

In [42]:
bird_file = open("yesbirdy.pkl","wb")
pickle.dump(birdy_dict,bird_file)
bird_file.close()

In [43]:
df_yesbirdy.to_csv('yesbirdy.csv', header=True)

****
# Human voices

In [44]:
#read csv of df_human
df_human = pd.read_csv(human_folder+'validated.tsv', sep='\t')

In [45]:
# separating males and females

all_rows = len(df_yesbirdy)
split = int(all_rows * 0.5)

df_male = df_human[df_human['gender'] == 'male'][:split] 
df_female = df_human[df_human['gender'] == 'female'][:split]
df_human = pd.concat([df_male, df_female])
df_human = shuffle(df_human, random_state = random_number).reset_index(drop=True)

print(len(df_yesbirdy))
print(len(df_human))
print(len(df_male))
print(len(df_female))


6045
6044
3022
3022


In [46]:
df_human.to_csv('human.csv', header=True)

In [47]:
import time
from IPython.display import Audio
from IPython.core.display import display

# for i in range(106,300):
#     f_name = warblr_folder + 'wav/' + df_yesbirdy.iloc[i]['itemid'] + '.wav'
#     librosa.load(f_name, sr=44100)
#     print(i)
#     display(Audio(f_name, autoplay = True, rate=44100))
# #     time.sleep(11)

In [48]:
data = df_human[df_human['path'] == list_of_mp3_names[0]]
#data['path'].values[0]

In [49]:
sr = librosa.get_samplerate('H://cv-corpus-7.0-2021-07-21/en/clips/'+list_of_mp3_names[0])

def add_human_data(row):
    #retrieving data and sample rate from file
    human, _ = librosa.load(hclip_folder+row, sr)
    
    return human

def add_samplerate(sr):
    #retrieving sample rate from file    
    return sr

#create function that retrieves data and sample rate from file
def create_human_dict(df):
    new_dict = {}
    for i in range(len(df)):
        new_dict[df['path'][i]] = add_human_data(df['path'][i])
    
    return new_dict

In [ ]:
sr

In [50]:
human_dict = create_human_dict(df_human)

C:\ProgramData\Anaconda3\lib\site-packages\librosa\core\audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


In [51]:
# create a binary pickle file 
human_file = open("human.pkl","wb")

# write the python object (dict) to pickle file
pickle.dump(human_dict,human_file)

# close file
human_file.close()

In [52]:
human_file.close()

***
# Experimental code underneath:
**not needed**

In [ ]:
# Audio(df_female['audio_array'][84], autoplay = True, rate=sr)

In [ ]:
# df_mixed = pd.concat([df_yesbirdy,df_human], axis=1)

In [ ]:
# df_mixed.to_csv('mixed.csv', index=False)
# 

In [ ]:
#normalization function

def normalization(array):
    """normalizes numpy arrays and returns values on a scale
    between -1 for the minumum value and 1 for the maximum value."""
    
    return 2*((array-array.min()) / (array.max()-array.min())) -1


 


In [ ]:
def random_mix(row):
    human_length = len(row['human_array'])
    bird_length = len(row['bird_array'])
    delta = abs(int(bird_length - human_length))
    
    if human_length <= bird_length:
        position = random.randrange(delta)
        return row['bird_array'][position:] + row['human_array']
    else:
        return row['bird_array'] + row['human_array'][:bird_length]
        

In [ ]:
def plot_spectrogram(data, samplerate):
    """ Plot spectrogram with mel scaling """
    
    sr = samplerate
    spectrogram = librosa.feature.melspectrogram(data, sr=sr)
    log_spectrogram = librosa.power_to_db(spectrogram, ref=np.max)
    librosa.display.specshow(log_spectrogram, sr=sr, x_axis='time', y_axis='mel')

plot_spectrogram(human, human_sr)

In [ ]:
# print(bird.shape)
# plot_spectrogram(mixed, bird_sr)

In [ ]:
def describe_array(array):
    print(np.min(array))
    print(np.max(array))
    print(np.mean(array))
    print(np.std(array))
    

In [ ]:
Audio(bird, autoplay = True, rate=sr)

In [ ]:
plt.plot(bird)